In [1]:
import numpy as np
import decimal
import matplotlib.pyplot as plt
import sympy as sp

In [2]:
def intializeContext(significantFigs,rounding) :
    #Intialize the context with specified signFigs and rounding/chopping
    decimal.getcontext().prec = significantFigs 
    if( rounding ) :
        decimal.getcontext().rounding = decimal.ROUND_HALF_UP
    else :
        decimal.getcontext().rounding = decimal.ROUND_DOWN 

In [3]:
def parse_exp(expr_str , var_name='x') :
    x = sp.symbols(var_name)
    expr = sp.sympify(expr_str)
    
    def decimal_func(x_decimal) :
        if not isinstance(x_decimal,decimal.Decimal) :
            x_decimal = decimal.Decimal(x_decimal)
            
        result = expr.evalf(decimal.getcontext().prec,subs={x:x_decimal})
        return decimal.Decimal(str(result))
    
    return decimal_func   

In [16]:
def parse_derivative(expr_str , var_name='x' ) :
    x = sp.symbols(var_name)
    expr = sp.sympify(expr_str)
    deriv = sp.diff(expr)
    
    def decimal_derv(x_decimal) :
        if not isinstance(x_decimal,decimal.Decimal) :
            x_decimal = decimal.Decimal(x_decimal)
        result = deriv.evalf(decimal.getcontext().prec,subs = {x:x_decimal})
        return decimal.Decimal(str(result))
    return decimal_derv
    

In [17]:
def NewtonRaphson(x0,func_expr,tol=1e-7 ,max_iter=100 , significantFigs = 14 , rounding = True):
    intializeContext(significantFigs,rounding=rounding)
    x_root = decimal.Decimal(x0)
    relativeError = decimal.Decimal("Infinity")
    func = parse_exp(func_expr)
    derivative = parse_derivative(func_expr)
    
    deriv_exp = str(sp.diff(sp.sympify(func_expr)))
    iteration_details = []
    iter = 0 
    for i in range(max_iter) :
        x_root_old = x_root
        x_root = x_root_old - (func(x_root_old) / derivative(x_root_old))
        if x_root != 0 and i != 0 :
            relativeError = abs(x_root-x_root_old)
            
        if func_expr != "" :
            expr_eval_str = f"{x_root_old}-(({func_expr.replace('x', str(x_root_old))})/ ({deriv_exp.replace('x', str(x_root_old))})) = {x_root}"
        else:
            expr_eval_str = str(x_root)    
            
        detials = {"iteration" : i+1,"x_root" : x_root , "Relative Error" : relativeError , "Evaluation" : expr_eval_str}
        
        iteration_details.append(detials)
        if relativeError <= tol :
            break 
    return x_root, iteration_details
        

In [19]:
root, table = NewtonRaphson(
    "1.5", 
    "x**2 -2", 
    tol=decimal.Decimal("1e-20"), 
    significantFigs=20
)
print("Root:", root)
for row in table:
    print(row)


Root: 1.4142135623730949780
{'iteration': 1, 'x_root': Decimal('1.4166666666666666667'), 'Relative Error': Decimal('Infinity'), 'Evaluation': '1.5-((1.5**2 -2)/ (2*1.5)) = 1.4166666666666666667'}
{'iteration': 2, 'x_root': Decimal('1.4142156862745097301'), 'Relative Error': Decimal('0.0024509803921569366'), 'Evaluation': '1.4166666666666666667-((1.4166666666666666667**2 -2)/ (2*1.4166666666666666667)) = 1.4142156862745097301'}
{'iteration': 3, 'x_root': Decimal('1.4142135623746899761'), 'Relative Error': Decimal('0.0000021238998197540'), 'Evaluation': '1.4142156862745097301-((1.4142156862745097301**2 -2)/ (2*1.4142156862745097301)) = 1.4142135623746899761'}
{'iteration': 4, 'x_root': Decimal('1.4142135623730951551'), 'Relative Error': Decimal('1.5948210E-12'), 'Evaluation': '1.4142135623746899761-((1.4142135623746899761**2 -2)/ (2*1.4142135623746899761)) = 1.4142135623730951551'}
{'iteration': 5, 'x_root': Decimal('1.4142135623730950584'), 'Relative Error': Decimal('9.67E-17'), 'Evalua